In [5]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math


/Users/amirkahriman/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [6]:
stocks = pd.read_csv('../Algorithmic-Python-Trading/static/sp-500-stocks/sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


Acquiring an API Token

In [7]:
from secret.secret import IEX_CLOUD_API_TOKEN

In [8]:

# symbol = 'AAPL'
# api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
api_url = f'https://api.iex.cloud/v1/data/core/quote/aapl?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data[0]['high']

In [9]:
symbol = data[0]['symbol']
price = data[0]['delayedPrice']
market_cap = data[0]['marketCap']
print(symbol)
print(price)
print(market_cap)

AAPL
None
2903621034640


In [10]:
my_columns = [ 'Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [11]:
# final_dataframe.append(
#     pd.Series(
#         [
#             symbol,
#             price,
#             market_cap,
#             'N/A'
#         ],
#         index = my_columns
#     ),
#     ignore_index = True
# )
new_row = pd.Series([symbol, price, market_cap, 'N/A'], index=my_columns)
final_dataframe = pd.concat([final_dataframe, pd.DataFrame([new_row])], ignore_index=True)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,None,2903621034640,N/A


In [12]:
final_dataframe = pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker']:
    api_url = f'https://api.iex.cloud/v1/data/core/quote/aapl?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    new_row = pd.Series([stock, data[0]['latestPrice'], market_cap, 'N/A'], index=my_columns)
    final_dataframe = pd.concat([final_dataframe, pd.DataFrame([new_row])], ignore_index=True)
final_dataframe

/var/folders/62/4whl7xzd4p3563p4ycn5xd780000gn/T/ipykernel_15764/1653065174.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe = pd.concat([final_dataframe, pd.DataFrame([new_row])], ignore_index=True)


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,186.695,2903621034640,N/A
1,AAL,186.695,2903621034640,N/A
2,AAP,186.695,2903621034640,N/A
3,AAPL,186.695,2903621034640,N/A
4,ABBV,186.695,2903621034640,N/A
...,...,...,...,...
500,YUM,186.750,2903621034640,N/A
501,ZBH,186.750,2903621034640,N/A
502,ZBRA,186.750,2903621034640,N/A
503,ZION,186.750,2903621034640,N/A


In [13]:
def chunks(lst, n):
    """Yield successive n-sized from list."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [14]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

final_dataframe = pd.DataFrame(columns= my_columns)
for symbol_string in symbol_strings:

    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        # Find the element with the matching symbol
        matching_symbol_data = next(item for item in data if item["symbol"] == symbol)
        # Extract the latestPrice
        latest_price = matching_symbol_data["latestPrice"]
        marketCap = matching_symbol_data["marketCap"]

        new_row = pd.Series([symbol, latest_price, marketCap, 'N/A'], index=my_columns)
        final_dataframe = pd.concat([final_dataframe, pd.DataFrame([new_row])], ignore_index=True)

final_dataframe


/var/folders/62/4whl7xzd4p3563p4ycn5xd780000gn/T/ipykernel_15764/2453265745.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe = pd.concat([final_dataframe, pd.DataFrame([new_row])], ignore_index=True)


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,139.440,40856491983,N/A
1,AAL,13.670,8933899319,N/A
2,AAP,63.520,3779566405,N/A
3,AAPL,186.790,2905098546080,N/A
4,ABBV,158.775,285795000000,N/A
...,...,...,...,...
500,YUM,129.480,36294308196,N/A
501,ZBH,121.845,25463254732,N/A
502,ZBRA,266.920,13708905500,N/A
503,ZION,44.405,6578547420,N/A


In [24]:
portfolio_size = input('Enter the value of your portfolio: ')

try: 
    val = float(portfolio_size)
except ValueError:
    print("That's not a valid number! Please try again:")
    portfolio_size = input('Enter the value of your portfolio: ')
    val = float(portfolio_size)

In [25]:
print("Previous value: ", val)
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Stock Price'][i])
final_dataframe

Previous value:  5000000.0


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,139.440,40856491983,71
1,AAL,13.670,8933899319,724
2,AAP,63.520,3779566405,155
3,AAPL,186.790,2905098546080,53
4,ABBV,158.775,285795000000,62
...,...,...,...,...
500,YUM,129.480,36294308196,76
501,ZBH,121.845,25463254732,81
502,ZBRA,266.920,13708905500,37
503,ZION,44.405,6578547420,222


In [44]:
writer = pd.ExcelWriter('Recommended-trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

In [45]:
background_color = '#0a0a23'
font_color = "#ffffff"

string_format = writer.book.add_format(
    {
        "font_color" : font_color,
        "bg_color" : background_color, 
        "border" : 1
    }
)

dollar_format = writer.book.add_format(
    {
        "num_format" : '$0.00',
        "font_color" : font_color,
        "bg_color" : background_color, 
        "border" : 1
    }
)

integer_format = writer.book.add_format(
    {
        "num_format" : '0',
        "font_color" : font_color,
        "bg_color" : background_color, 
        "border" : 1
    }
)

In [46]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
# writer.close()

writer.sheets['Recommended Trades'].write('A1', 'Tricker', string_format)
writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', dollar_format)
writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', integer_format)

0

In [47]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}
for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

writer.close()